In [ ]:
from utility import *

# Einlesen der Daten

In [8]:
mitte = [load_counts('../messungen/mitte/Cal_2020-01-10_09-24-T.txt'),
load_counts('../messungen/mitte/Cal_2020-01-10_09-24-EdetA.txt'),
load_counts('../messungen/mitte/Cal_2020-01-10_09-24-EdetB.txt')]


In [61]:
channels_t = np.arange(1, mitte[0].size + 1)
channels_e = np.arange(1, mitte[1].size + 1)
energies_a = energy_a(channels_e)
energies_b = energy_b(channels_e)


# Plot ueber Energie

In [78]:
%matplotlib qt5
fig, ax = set_up_plot()
ax.set_xlabel('Energie [keV]')
ax.set_ylabel('Normierte Zaehlrate')
ax.step(energies_a, mitte[1]/mitte[1].max())
ax.step(energies_b, mitte[2]/mitte[2].max())
save_fig(fig, 'mid_over_energy', 'calibration')

# Plot ueber Energie

In [73]:
%matplotlib qt5
fig, ax = set_up_plot()
ax.step(channels_e, mitte[1]/mitte[1].max())
ax.step(channels_e, mitte[2]/mitte[2].max())
save_fig(fig, 'mid_over_channel', 'calibration')

In [67]:
range_b = [1600, 2300]
range_a = [2800, 3600]


In [70]:
fig, ax = set_up_plot()
ax.step(channels_t, mitte[0])

In [71]:
range_t = [970, 1140]